# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


/Users/henriquebueno/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/models-master/research/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/henriquebueno/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/henriquebueno/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/henriquebueno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/henriquebueno/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/henriquebuen

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
#alteraçoes henrique aqui

# What model to download.
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_NAME = 'celular_inference_graph'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')

#NUM_CLASSES = 90
NUM_CLASSES = 1

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
#print(categories)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [8]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = 'test_images'
PATH_TO_TEST_IMAGES_DIR = '/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/find_phone_data'
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i)) for i in [0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,57,58,59,60,61,62,63,64,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134] ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
#IMAGE_SIZE = (490, 326)

In [9]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [10]:
import math

#aaa
def temTelefone(detection_boxes1, detection_classes1, detection_scores1, fileName):

    xEstimado = -1
    yEstimado = -1
    for pos in range(len(detection_classes1)):
        #print("-----------classe: " + str(detection_classes1[pos]))
        #if detection_classes1[pos]==77:#classe cell phone
        if detection_classes1[pos]==1:#classe celular
            #print("Nome do arquivo: " + str(filename))
            coordenadas = detection_boxes1[pos]
            #print("Posicao do celular encontrado: " + str(coordenadas))
            ymin = coordenadas[0]
            xmin = coordenadas[1]
            ymax = coordenadas[2]
            xmax = coordenadas[3]
            #print("Y min: " + str(ymin))
            #print("X min: " + str(xmin))
            #print("Y max: " + str(ymax))
            #print("X max: " + str(xmax))
            xEstimado=(xmin+xmax)/2
            yEstimado=(ymin+ymax)/2
            #xCorreto0=0.8306 
            #yCorreto0=0.1350

            #TAMBÉM PRECISA ALTERAR A CELULA DETECTION--------------------------

            #Nao achou para a 1.jpg
            #xCorreto1=0.8714
            #yCorreto1=0.1718

            #xCorreto3=0.6857
            #yCorreto3=0.6933

            #print("Ponto: " + str(xEstimado) + " - " + str(yEstimado))
            #print("Score do celular encontrado: " + str(detection_scores1[pos]))
            #dist = math.hypot(xEstimado - xCorreto3, yEstimado - yCorreto3)
            #print("Distancia: " + str(dist))
            
    return xEstimado, yEstimado

In [11]:
#carrega o arquivo com o nome dos arquivos e as coordenadas dos celulares em cada
#arquivo em uma matriz onde cada linha é um registro do arquivo labels.txt e cada
#linha tem 3 valores: NOME_ARQUIVO X Y
#ATENCAO: os 3 valores de cada linha sao armazenados como string

def carregaArquivo(arquivo):

    #abre o arquivo
    file = open(arquivo, "r")

    #le todas as linhas
    linhas=file.readlines()
    
    #fecha arquivo
    file.close()

    return linhas

In [ ]:
#Funcao que recebe como entrada um array com os registros de entrada e um
#string com o NOME_ARQ. O retorno é a posicao do arquivo NOME_ARQ em registros.
#Caso ele não encontre o nome, ele retorna -1
def localizaArquivoEmLinhas(linhas, nomeArquivo):
    retorno = -1
    
    pos=0
    for linha in linhas:
        temp = linha.split()
        nome=temp[0]
        if(nome==nomeArquivo):
            return pos
        pos+=1
    
    return retorno

In [ ]:
#predicao

arquivo = "/Users/henriquebueno/DOUTORADO/2018.01/ml/trabalho2/labels.txt"
linhas = carregaArquivo(arquivo)
#pos = localizaArquivoEmLinhas(linhas, "129.jpg")
#print(pos)
#linha = linhas[pos]
#linhaTokenizada = linha.split()
#print(linhaTokenizada)

sim = 0
nao = 0
simComDistancia=0

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  #print("abriu " + str(image.filename))
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  #print("ja rodou inferencia " + str(image.filename))
  # Visualization of the results of a detection.
  #vis_util.visualize_boxes_and_labels_on_image_array(
  #    image_np,
  #    output_dict['detection_boxes'],
  #    output_dict['detection_classes'],
  #    output_dict['detection_scores'],
  #    category_index,
  #    instance_masks=output_dict.get('detection_masks'),
  #    use_normalized_coordinates=True,
  #    line_thickness=8)
  #print("vis_util.visualize_boxes_and_labels_on_image_array " + str(image.filename))
  #plt.figure(figsize=IMAGE_SIZE)
  #print("plt.figure " + str(image.filename))
  #plt.imshow(image_np)
  #print("plt.imshow " + str(image.filename))

  detection_boxes1 = output_dict['detection_boxes']
  detection_classes1 = output_dict['detection_classes']
  detection_scores1 = output_dict['detection_scores']
    
  xEstimado, yEstimado = temTelefone(detection_boxes1, detection_classes1, detection_scores1, image.filename)

  if((xEstimado==-1)and(yEstimado==-1)):
    #print("Não: " + str(image.filename))
    print("NAO")
    nao+=1
  else:
    pos = localizaArquivoEmLinhas(linhas, image.filename)
    linha = linhas[pos]
    linhaTokenizada = linha.split()
    xCorreto=float(linhaTokenizada[1])
    yCorreto=float(linhaTokenizada[2])
    
    dist = math.hypot(xEstimado - xCorreto, yEstimado - yCorreto)    
    if(dist<0.05):
        simComDistancia+=1
    
    #print("Sim: " + str(image.filename) + " x, y= " + str(xEstimado) + "-" + str(yEstimado))
    print("SIM " + str(dist))
    sim+=1
    
#ainda preciso calcular a distancia do celular encontrado para a coordenada do arquivo do professor
    
print("% sim: " + str(sim/(sim+nao)))
print("% nao: " + str(nao/(sim+nao)))
print("% sim com distancia < 0.05: " + str(simComDistancia/(sim+nao)))

  
  
  

  
        

SIM 0.6058863245054297
SIM 0.5846966307888274
SIM 0.27558821898666785
SIM 0.5891126400630563
SIM 0.09777259130480354
SIM 0.4779666912286538
SIM 0.3660706401931364
SIM 0.5576292692697307
SIM 0.2390820995348739
SIM 0.4092688776542116
SIM 0.22165478559902982
SIM 0.4468810979652958
SIM 0.5949942815412954
SIM 0.564483105214849
SIM 0.46881805448297986
SIM 0.2597533168259244
SIM 0.5737231070683728
SIM 0.6835426546174934
SIM 0.3196092825651829
SIM 0.5613775050320421


/Users/henriquebueno/anaconda/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


SIM 0.5985568283529286
SIM 0.6817362661480848
SIM 0.4307902606702924
SIM 0.7557153789431008
SIM 0.4570023835500732
SIM 0.7128479770872033
SIM 0.5204628904347074
SIM 0.4566993279512329
SIM 0.7791689536670682
SIM 0.4390375042323532
SIM 0.6795496122892716
SIM 0.76626224595965
SIM 0.7590890448357327
SIM 0.13348070106811188
SIM 0.57844709224016
SIM 0.5754436960572068
SIM 0.4022319645298919
SIM 0.4832221663089888
SIM 0.1996153019251028
SIM 0.4247066701962281
SIM 0.3901463478757484
SIM 0.2627234889232992
SIM 0.14422514417100468
SIM 0.4236256077207574
SIM 0.2059779719812429
SIM 0.46080368791912546
SIM 0.6664381850235646


KeyboardInterrupt: 

In [ ]:
print("acabou")